In [86]:
import os # file system stuff
import pandas as pd #data frames
import math #sqrt
import numpy as np #arrays

In [87]:
def search_str(linesObj,word,searchStart=0,ignore=0):
    #lines obj - list, each containing a string. e.g. the output of file.readlines() method
    #word - string that we look to match
    #searchStart - integer, if we don't need to start searching at the beginning, choose \
    #line number to start at
    #ignore - integer on how many instances of the string to ignore 
    #(e.g. if you need the second instance, ignore=1)
    #Returns index of line that word occurs, or -1 if not found
    wordLine=-1 #will return -1 if string not found
    #start for
    for ln_num, line in enumerate(linesObj): #iterate over lines
        #print(line)
        
        if line.find(word) >-1 and linesObj.index(line) >= searchStart:
        # start outer if - True if line found past the searchStart    
            if ignore == 0:
            #Start inner if - if this is the instance of the string you want, get index    
                wordLine = ln_num
                break
            else:
            #if not instance of string you want, deduct from the count    
                ignore = ignore - 1
                continue
            #end inner if
       #end outer if     
    #end for            
    return wordLine

In [88]:
def get_di_table(data,tableHeader='2*D2(A,B)          DI(A,B)          %Deloc(A,B)       %Deloc(B,A)',endString='Areas of Interatomic Surfaces:'):
    tableStart = search_str(data,tableHeader,searchStart=0)+2 #data starts 2 lines after header
    tableEnd = search_str(data,endString,tableStart)-2 #-2 since there is a 2 line gap
    tableData=data[tableStart:tableEnd]
    headerNames = ['Atom A','Atom B']
    for colName in tableHeader.split():
        headerNames.append(colName) 
    table = pd.DataFrame(columns=headerNames)
    #Add each row of table to Data Frame
    for i,line in enumerate(tableData):
        print(line.split())
        table.loc[i] = line.split()[0:6]
    return table        

In [89]:
def get_table(data,tableHeader,ignored=0,endString='Total'):
    #data - list each containing a string. i.e. the output of sumfile.readlines() method
        #note this function is designed for .sum files and won't work on others
    #tableHeader - the header of the table, NOT including "Atom" or "Atom A"
    #ignored=0 - to be passed to search_str, find the (ignored)+1 instance of the string
    #endString - a string two past the last row where data in the table is
    #For most .sum file tables, this is Total, but for some (quadrupoles)
    #, there is no Total, so it will usually take the name of the next table
    #returns a DataFrame with Table matching the header
    #Find start and end lines of table
    tableStart = search_str(data,tableHeader,searchStart=0,ignore=ignored)+2 #data starts 2 lines after header
    tableEnd = search_str(data,endString,tableStart)-1 #only deduct 1, because next line will get up to but not including that line
    #So the -1 here and the next line combine to get data up to two rows before endString
    tableData=data[tableStart:tableEnd] #get data
    #Generate column names of DataFrame. #Include atom here, as sumfile header is Atom A and the whitespace there will mess with how code is writting
    headerNames = ['Atom']
    for colName in tableHeader.split():
        headerNames.append(colName)
    #Initialize empty data frame    
    table = pd.DataFrame(columns=headerNames)
    #Add each row of table to Data Frame
    for i,line in enumerate(tableData):
        table.loc[i] = line.split()
        
    return table    

In [90]:
def get_bcp_block(data,atPair=["C1","H2"]):
    #find all the lines with BCP data for a pair of atoms
    #data - list each containing a string. i.e. the output of sumfile.readlines() method
        #note this function is designed for .sum files and won't work on others
    #atPair - list of length 2, containing atom element and numeric label, see default for ex    
    #returns list of strings with BCP data    
    #find start line of BCP data, search both permutations of atom labels    
    #Start -1 from the search line, because the coords are the line before
    bcpStart = search_str(data,word='Type = (3,-1) BCP {at1} {at2}'.format(at1=atPair[0],at2=atPair[1]))-1
    if bcpStart == -2:
        print('didnt find first')
        bcpStart = search_str(data,word='Type = (3,-1) BCP {at1} {at2}'.format(at1=atPair[1],at2=atPair[0]))-1
    bcpEnd = search_str(data,word='GBL_IV =',searchStart=bcpStart) #find end of bcp data at GBL_IV
    bcpBlock = data[bcpStart:bcpEnd]
    return bcpBlock # return the lines of the BCP data

In [91]:
def get_sub_di(data,subAtomLabels=[]):
    diTable = get_di_table(data)
    diTable = diTable.drop(['2*D2(A,B)','%Deloc(A,B)','%Deloc(B,A)'],axis=1)
    print(diTable)
    diClass = []
    #logic error - need to instead just iterate over tables and see if equals any of subAtomLabels for A/B
    #after meeting
    diTable['DI(A,B)'] = diTable['DI(A,B)'].astype(float)
    for ind in diTable.index:
        if any(x == diTable['Atom A'][ind] for x in subAtomLabels) and any(x == diTable['Atom B'][ind] for x in subAtomLabels):
            diClass.append('Substituent-Substituent')
        elif any(x == diTable['Atom A'][ind] for x in subAtomLabels) or any(x == diTable['Atom B'][ind] for x in subAtomLabels):
            diClass.append('Substituent-Substrate')
        else:
            diClass.append('Substrate-Substrate')
    diTable['Interaction'] = diClass
    return sum(diTable.loc[diTable['Interaction'] == 'Substituent-Substrate', 'DI(A,B)'])

In [93]:
def get_bcp_properties(bcpBlock):
    #bcpBlock: takes output of get_bcp_block
    #initialize empyt dict
    bcpDict={}
    for line in bcpBlock: #iterate over lines
        splitLine = line.split() #split the line into individual words based on whitespace
        #note splitLine[i] - i is from manual inspection of data
        if 'Coords' in splitLine: #if line contains the coordinates, get coordinates
            bcpDict.update({'xyz': np.array([float(splitLine[4]),float(splitLine[5]),float(splitLine[6])])})
            #bcpDict.update({'y': splitLine[5]})
            #bcpDict.update({'z': splitLine[6]})
        elif 'Rho' in splitLine: #get density at BCP
            bcpDict.update({'rho': [float(splitLine[2])]})
        elif 'HessRho_EigVals' in splitLine: #get lambda_i
            bcpDict.update({'lambda1': [float(splitLine[2])]})
            bcpDict.update({'lambda2': [float(splitLine[3])]})
            bcpDict.update({'lambda3': [float(splitLine[4])]})
        elif 'DelSqRho' in splitLine: #get DelSqRho
            bcpDict.update({'DelSqRho': [float(splitLine[2])]})
        elif 'Ellipticity' in splitLine: #get ellipticity
            bcpDict.update({'Ellipticity': [float(splitLine[3])]})
        elif 'V' in splitLine: #get V
            bcpDict.update({'V': [float(splitLine[2])]})
        elif 'G' in splitLine: # get G
            bcpDict.update({'G': [float(splitLine[2])]})
        #elif line.find('Rho') 
    bcpDict.update({'H': [bcpDict['V'][0] + bcpDict['G'][0]]})  #get total energy density  
    return bcpDict

In [94]:
def get_atomic_props(data):
    #data - - list each containing a string. i.e. the output of sumfile.readlines() method
        #note this function is designed for .sum files and won't work on others
    #returns a dictionary of dictionaries. The keys for the outer dictionary are atom labels
    #the keys for the inner dictionary are properties
    #e.g. can get data as allatomDict['C1']['Mu_X'], which gets x dipole for atom C1
    
    #find tables where desired data is located    
    xyzFrame = get_table(data,'Charge                X                  Y                  Z',endString='Some Atomic Properties:')
    eFrame = get_table(data,'q(A)              L(A)              K(A)          K_Scaled(A)      |Mu_Intra(A)|')
    muXFrame = get_table(data,'Mu_Intra_X(A)     Mu_Bond_X(A)        Mu_X(A)')
    muYFrame = get_table(data,'Mu_Intra_Y(A)     Mu_Bond_Y(A)        Mu_Y(A)')
    muZFrame = get_table(data,'Mu_Intra_Z(A)     Mu_Bond_Z(A)        Mu_Z(A)')
    muMagFrame = get_table(data,'|Mu_Intra(A)|     |Mu_Bond(A)|        |Mu(A)|')
    quadFrame = get_table(data,'Q_XX(A)           Q_XY(A)           Q_XZ(A)           Q_YY(A)           Q_YZ(A)           Q_ZZ(A)',endString='Eigenvalues and Eigenvectors of Atomic Traceless Quadrupole Moment Tensors')
    #need these radial distoritions for calculating atomic quadrupole contributions
    radFrame = get_table(data,'R-2(A)            R-1(A)            R0(A)             R+1(A)            R+2(A)',endString='Atomic Radial Distortion Moments')
    #currently volume is getting the 0.0004 isosurface, and i don't know how to change that
    #like in theory this should work, but it's not. I'll fix later
    volFrame = get_table(data,'Area(A)           Vol(A)          N(Vol(A))      N(Vol(A))/Vol(A)     %N(Vol(A))',ignored=1)
    
    #atomicFrame = pd.concat([eFrame,muXFrame,muYFrame,muZFrame,quadFrame,volFrame,radFrame],axis=1)
    allatomDict = {} #initialize empty dictionary to store dictionaries of the atoms
    for atom in eFrame['Atom']:
        atomDict = {} #create a dictionary for each atom
        #the following lines go through the tables, and find the row and column with desired property
        # the row matches Atom, the column matches property we want, and we use .iloc[0] to get it as a string, which we then change to float. 
        #If no iloc[0] it is a data frame object and float will eventually return an error on it
        #xyz is a np array - element 0 is x, element 1 is y, element 2 is z
        atomDict.update({'xyz': np.array([float(xyzFrame[xyzFrame['Atom'] == atom]['X'].iloc[0]),float(xyzFrame[xyzFrame['Atom'] == atom]['Y'].iloc[0]),float(xyzFrame[xyzFrame['Atom'] == atom]['Z'].iloc[0])])})
        atomDict.update({'q': [float(eFrame[eFrame['Atom'] == atom]['q(A)'].iloc[0])]})
        atomDict.update({'K': [float(eFrame[eFrame['Atom'] == atom]['K(A)'].iloc[0])]})
        atomDict.update({'K_Scaled': [float(eFrame[eFrame['Atom'] == atom]['K_Scaled(A)'].iloc[0])]})
        atomDict.update({'Mu_Intra_X': [float(muXFrame[muXFrame['Atom']==atom]['Mu_Intra_X(A)'].iloc[0])]})
        atomDict.update({'Mu_Intra_Y': [float(muYFrame[muYFrame['Atom']==atom]['Mu_Intra_Y(A)'].iloc[0])]})
        atomDict.update({'Mu_Intra_Z': [float(muZFrame[muZFrame['Atom']==atom]['Mu_Intra_Z(A)'].iloc[0])]})
        atomDict.update({'Mu_Bond_X': [float(muXFrame[muXFrame['Atom']==atom]['Mu_Bond_X(A)'].iloc[0])]})
        atomDict.update({'Mu_Bond_Y': [float(muYFrame[muYFrame['Atom']==atom]['Mu_Bond_Y(A)'].iloc[0])]})
        atomDict.update({'Mu_Bond_Z': [float(muZFrame[muZFrame['Atom']==atom]['Mu_Bond_Z(A)'].iloc[0])]})
        atomDict.update({'Mu_X': [float(muXFrame[muXFrame['Atom']==atom]['Mu_X(A)'].iloc[0])]})
        atomDict.update({'Mu_Y': [float(muYFrame[muYFrame['Atom']==atom]['Mu_Y(A)'].iloc[0])]})
        atomDict.update({'Mu_Z': [float(muZFrame[muZFrame['Atom']==atom]['Mu_Z(A)'].iloc[0])]})
        atomDict.update({'|Mu_Intra|': [float(muMagFrame[muMagFrame['Atom']==atom]['|Mu_Intra(A)|'].iloc[0])]})
        atomDict.update({'|Mu_Bond|': [float(muMagFrame[muMagFrame['Atom']==atom]['|Mu_Bond(A)|'].iloc[0])]})
        atomDict.update({'|Mu|': [float(muMagFrame[muMagFrame['Atom']==atom]['|Mu(A)|'].iloc[0])]})
        atomDict.update({'Q_XX': [float(quadFrame[quadFrame['Atom']==atom]['Q_XX(A)'].iloc[0])]})
        atomDict.update({'Q_XY': [float(quadFrame[quadFrame['Atom']==atom]['Q_XY(A)'].iloc[0])]})
        atomDict.update({'Q_XZ': [float(quadFrame[quadFrame['Atom']==atom]['Q_XZ(A)'].iloc[0])]})
        atomDict.update({'Q_YY': [float(quadFrame[quadFrame['Atom']==atom]['Q_YY(A)'].iloc[0])]})
        atomDict.update({'Q_YZ': [float(quadFrame[quadFrame['Atom']==atom]['Q_YZ(A)'].iloc[0])]})
        atomDict.update({'Q_ZZ': [float(quadFrame[quadFrame['Atom']==atom]['Q_ZZ(A)'].iloc[0])]})
        atomDict.update({'R+2': [float(radFrame[radFrame['Atom']==atom]['R+2(A)'].iloc[0])]})
        atomDict.update({'R+1': [float(radFrame[radFrame['Atom']==atom]['R+1(A)'].iloc[0])]})
        atomDict.update({'Vol': [float(volFrame[volFrame['Atom']==atom]['Vol(A)'].iloc[0])]})
        allatomDict.update({atom: atomDict}) #add the atomic dictionary to the dictionary of dictionaries    
    return allatomDict

In [95]:
def get_dist(coordListA,coordListB):
    #takes 2 points, coordList are 3 element list/array of xyz coords.
    #returns distance between them
    dist = math.sqrt((coordListA[0]-coordListB[0])**2 + (coordListA[1]-coordListB[1])**2 + (coordListA[2]-coordListB[2])**2)
    return dist

In [96]:
def find_closest_nuclei(xyz,atomDict):
    #probably need to update in symmetry case in case of equidistant nuclei
    # xyz - takes a point as a 3 length list of xyzcoords
    # atomDict - object from get_atomic_props
    #returns the two atoms that are closest to the xyz point given
    # original design purpose was to find nuclei closest to a charge concentration
    distList=[] #initialize empty list for distances
    atList = [] #initialize empty list for atom labels
    for atom in atomDict: #get list of distances from point xyz to all atoms, and list of atom labels
        atList.append(atom)
        distList.append(get_dist(xyz,atomDict[atom]['xyz']))
    npDistList= np.array(distList) #convert distance list to array for np.where functionality
    minDist = np.partition(npDistList,1)[0:2] #find the two lowest distances
    #return atList with indices of npDistList that are equal to the two lowest values
    return [atList[np.where(npDistList==minDist[0])[0][0]],atList[np.where(npDistList==minDist[1])[0][0]]]

In [97]:
def identify_vscc(multiccDict,atomDict, thresh=0.7):
    #multiccDict - dictionary of charge concentrations - output of get_cc_props
    #atomDict - dictionary of atomic properties - output of get_atomic_props
    #thresh - numeric threshold for distance between shells 
    #(eg inner shell charge concentration is more than 0.7 au closer to nuclei than VSCC)
    
    #takes a list of all charge concentrations
    #filters it to return dictionary of only points identified as VSCC based on below criteria
    #will be a vscc if its the farthest out, if its not on a bond line, and if it is not close to nuclei
    
    vsccDict={} #initialize empty dictionary for cc
    nucDistList = [] #empty list to store distances
    potentialccList = [] #empty list to store keys for potential VSCC after eliminating some criteria
    for cc in multiccDict: #for each cc
        if multiccDict[cc]['distFromNuc'] > 0.1: # this will filter out the innermost cc that are on the nuclei
            ccXYZ = multiccDict[cc]['xyz'] #get the xyz coords of the cc
            nucPair = find_closest_nuclei(ccXYZ,atomDict) #find the two closest atoms to the cc
            #if it is a bonded cc, it will be the nuclei to which it is bonded
            #check if the cc is on the bond between those two nuclei
            isBonded = is_on_line(atomDict[nucPair[0]]['xyz'],atomDict[nucPair[1]]['xyz'],ccXYZ)
            if not isBonded: #if it is not on the line, it is potentially a VSCC, store it, and its distance
                nucDistList.append(multiccDict[cc]['distFromNuc'])
                potentialccList.append(cc)
    #at this point, non-bonded, non-core CC remain, so just need to check which CC are the outermost
    #do this by comparing to maximum distance from nuclei
    if len(nucDistList) > 0:
        outerShellDist = max(nucDistList) #note this only checks nonbonded noncore
    for cc in potentialccList:
        if abs(multiccDict[cc]['distFromNuc']-outerShellDist) < thresh: 
            #for given cc, if close (within thresh) to outermost, it is a VSCC, store it as such
            #So, going up to 3rd row for now. May work for 4th but I haven't checked
            #C/O/F easy - any not core not bonded should be VSCC
            #P/S/Si harder
            # in example data inner~0.25-0.3, outer ~ 1.2-1.3
            #arbitrary default threshold seems like 0.7 will work
            vsccDict.update({cc: multiccDict[cc]})
    return vsccDict    

In [98]:
def is_on_line(lineStartList,lineEndList,pointToCheckList,epsilon=0.1):
    #return boolean if pointToCheck is on bond between lineStart and lineEnd
    #takes 3 lists of coordinates of 3 length xyz
    #line connecting atoms: (x1 + t(x2-x1),y1 + t(y2-y1),z1 + t(z2-z1))
    #find t - written out on a piece of paper somewhere based on dot product of vectors
    #to reconstruct - create equations of lines. (pointToCheck-pointWitht) and (lineStart-pointWitht)
    # dot product of those vectors should be 0(closest point is. 90 degrees)
    #rearrange to solve for t
    t = ((pointToCheckList[0]-lineStartList[0])*(lineEndList[0]-lineStartList[0]) + 
        (pointToCheckList[1]-lineStartList[1])*(lineEndList[1]-lineStartList[1]) + 
        (pointToCheckList[2]-lineStartList[2])*(lineEndList[2]-lineStartList[2]))/(((lineEndList[2]-lineStartList[2])**2) + ((lineEndList[1]-lineStartList[1])**2)+((lineEndList[0]-lineStartList[0])**2))
    #Find the point defined by t
    perpPoint = np.array([lineStartList[0]+t*(lineEndList[0]-lineStartList[0]),
                          lineStartList[1]+t*(lineEndList[1]-lineStartList[1]),
                          lineStartList[2]+t*(lineEndList[2]-lineStartList[2])])
    #get distance between test point and closest line point
    distToLine = get_dist(pointToCheckList,perpPoint) 
    #define thresholds to check if t lies between nuclei or beyond.
    #note - can I just check if t < 0 or t > 1 for that?
    #leaving as is for now
    lowX = min(lineStartList[0],lineEndList[0])
    highX = max(lineStartList[0],lineEndList[0])
    lowY = min(lineStartList[1],lineEndList[1])
    highY = max(lineStartList[1],lineEndList[1])
    lowZ = min(lineStartList[2],lineEndList[2])
    highZ = max(lineStartList[2],lineEndList[2])
    #if t is between the end points of the line, and the pointToCheck is within epsilon of that point, it is a bonded cc
    #return boolean based on evaluation
    if perpPoint[0] >= lowX and perpPoint[0] <= highX and perpPoint[1] >= lowY and perpPoint[1] <= highY and perpPoint[2] >= lowZ and perpPoint[2] <= highZ and distToLine < epsilon:
        return True
    else:
        return False

In [99]:
def get_cc_props(filename,atomLabel,type='vscc'):
    #takes a sumfilename with no extension and an atom label that we want cc for (eg F4)
    #returns dictionary of all cc for atomLabelwith all the (3,+3) cc and properties
    #create path to subdirectory
    currdir = os.getcwd()
    #example dir name SubH_CCF-ReorPosY-B3LYP-def2-TZVPPD-Field_atomicfiles
    subdirname = filename + "_atomicfiles"
    pathToSubdir = currdir+ "/"+subdirname+"/"
    lowerAtomLabel = atomLabel.lower() #the laprhocp files are eg f4.agpviz - note lower case
    #open the file, read data, and close file
    atFile = open(pathToSubdir+lowerAtomLabel+".agpviz",'r')
    atData = atFile.readlines()
    atFile.close()
    #initialize empty dict to store all cc
    allccDict = {}
    alldictcounter=1 #counter since the key will be numeric
    
    for lnNum,line in enumerate(atData):
        if "Type = (3,+3)" in line: #if we're looking at CC
            oneccDict = {} # create empty dict for storing properties for one cc
            xyzSplit = atData[lnNum+1].split() #line after label is xyz, split it and store xyz
            oneccDict.update({'xyz':np.array([float(xyzSplit[2]),float(xyzSplit[3]),float(xyzSplit[4])])})
            #next line is distance from nuc, split and store
            distFromNucSplit = atData[lnNum+2].split()
            oneccDict.update({'distFromNuc': float(distFromNucSplit[2])})
            #next is rho, then delsqrho - split and store those
            rhoSplit = atData[lnNum+3].split()
            oneccDict.update({'rho': float(rhoSplit[2])})
            delsqrhoSplit = atData[lnNum+4].split()
            oneccDict.update({'delsqrho': float(delsqrhoSplit[2])})
            #add the cc to the dictionary, update counter
            allccDict.update({alldictcounter: oneccDict})
            alldictcounter += 1
    #find all (3,+3) cps and get their properties
    return allccDict

In [100]:
def get_atomic_quad_contrib(atomDict):  
    #atomDict - one of the dictionary object returned from get_atomic_props
    secondMomentDict = {}
    #calculate the second moment based on the moments from dictionary
    #return dictionary
    secondMomentDict.update({'Qxx': atomDict['q'][0]*(atomDict['xyz'][0]**2) + 
                            atomDict['Q_XX'][0] + atomDict['R+2'][0]/3 + atomDict['xyz'][0]*atomDict['Mu_Intra_X'][0] + 
                            atomDict['xyz'][0]*atomDict['Mu_Intra_X'][0]})
    secondMomentDict.update({'Qxy': atomDict['q'][0]*atomDict['xyz'][0]*atomDict['xyz'][1] + 
                            atomDict['Q_XY'][0]+ atomDict['xyz'][1]*atomDict['Mu_Intra_X'][0] + 
                            atomDict['xyz'][0]*atomDict['Mu_Intra_Y'][0]})
    secondMomentDict.update({'Qxz': atomDict['q'][0]*atomDict['xyz'][0]*atomDict['xyz'][2] + 
                            atomDict['Q_XZ'][0]+ atomDict['xyz'][2]*atomDict['Mu_Intra_X'][0] + 
                            atomDict['xyz'][0]*atomDict['Mu_Intra_Z'][0]})
    secondMomentDict.update({'Qyy': atomDict['q'][0]*(atomDict['xyz'][1]**2) + 
                            atomDict['Q_YY'][0] + atomDict['R+2'][0]/3+ atomDict['xyz'][1]*atomDict['Mu_Intra_Y'][0] + 
                            atomDict['xyz'][1]*atomDict['Mu_Intra_Y'][0]})
    secondMomentDict.update({'Qyz': atomDict['q'][0]*atomDict['xyz'][1]*atomDict['xyz'][2] + 
                            atomDict['Q_YZ'][0]+ atomDict['xyz'][2]*atomDict['Mu_Intra_Y'][0] + 
                            atomDict['xyz'][1]*atomDict['Mu_Intra_Z'][0]})
    secondMomentDict.update({'Qzz': atomDict['q'][0]*(atomDict['xyz'][2]**2) + 
                            atomDict['Q_ZZ'][0] + atomDict['R+2'][0]/3 + atomDict['xyz'][2]*atomDict['Mu_Intra_Z'][0] + 
                            atomDict['xyz'][2]*atomDict['Mu_Intra_Z'][0]})
    secondMomentDict.update({'trace': secondMomentDict['Qxx'] + secondMomentDict['Qyy']+secondMomentDict['Qzz']})
    atomicQuadrupoleDict = {}
    #get the atomic contribution to quadrupole
    atomicQuadrupoleDict.update({'Q_xx': [0.5*(3*secondMomentDict['Qxx']-secondMomentDict['trace'])]})
    atomicQuadrupoleDict.update({'Q_xy': [0.5*(3*secondMomentDict['Qxy'])]})
    atomicQuadrupoleDict.update({'Q_xz': [0.5*(3*secondMomentDict['Qxz'])]})
    atomicQuadrupoleDict.update({'Q_yy': [0.5*(3*secondMomentDict['Qyy']-secondMomentDict['trace'])]})
    atomicQuadrupoleDict.update({'Q_yz': [0.5*(3*secondMomentDict['Qyz'])]})
    atomicQuadrupoleDict.update({'Q_zz': [0.5*(3*secondMomentDict['Qzz']-secondMomentDict['trace'])]})
    return atomicQuadrupoleDict  #return dictionary

In [101]:
def get_sub_props(atomDict,subAtoms,atomList):
    # atom dict - object from get_atomic_props, after updating with get_atomic_quad_contrib - ALL atom 
    #subAtoms - indices of atoms in group. Starting at 1
    #    atomList - list of atom labels for all atoms in dictionary 
    # returns dictionary of substituent properties
    groupDict={} #create empty dict
    first=True #flag for first iteration through loop
    for atom in subAtoms:
        if first: #if first time, create all elements in dicionary
            first=False #don't come here again
            #could do this with a for loop, but only want select keys, not all
            #add the atomic property to the dictionary - at this point just atomic, will update later
            groupDict = {'q': [atomDict[atomList[atom-1]]['q'][0]], 
                         'K': [atomDict[atomList[atom-1]]['K'][0]],
                         'K_Scaled': [atomDict[atomList[atom-1]]['K_Scaled'][0]],
                         'Mu_Intra_X': [atomDict[atomList[atom-1]]['Mu_Intra_X'][0]],
                         'Mu_Intra_Y': [atomDict[atomList[atom-1]]['Mu_Intra_Y'][0]],
                         'Mu_Intra_Z': [atomDict[atomList[atom-1]]['Mu_Intra_Z'][0]],
                         'Mu_Bond_X': [atomDict[atomList[atom-1]]['Mu_Bond_X'][0]],
                         'Mu_Bond_Y': [atomDict[atomList[atom-1]]['Mu_Bond_Y'][0]],
                         'Mu_Bond_Z': [atomDict[atomList[atom-1]]['Mu_Bond_Z'][0]],
                         'Mu_X': [atomDict[atomList[atom-1]]['Mu_X'][0]],
                         'Mu_Y': [atomDict[atomList[atom-1]]['Mu_Y'][0]],
                         'Mu_Z': [atomDict[atomList[atom-1]]['Mu_Z'][0]],
                         'Q_xx': [atomDict[atomList[atom-1]]['quadContrib']['Q_xx'][0]],
                         'Q_xy': [atomDict[atomList[atom-1]]['quadContrib']['Q_xy'][0]],
                         'Q_xz': [atomDict[atomList[atom-1]]['quadContrib']['Q_xz'][0]],
                         'Q_yy': [atomDict[atomList[atom-1]]['quadContrib']['Q_yy'][0]],
                         'Q_yz': [atomDict[atomList[atom-1]]['quadContrib']['Q_yz'][0]],
                         'Q_zz': [atomDict[atomList[atom-1]]['quadContrib']['Q_zz'][0]],
                         'Vol': [atomDict[atomList[atom-1]]['Vol'][0]]}
        else: #for the rest, add the atomic property to the group dictionary element
            for prop in groupDict:
                if 'Q' not in prop:
                    groupDict[prop][0] += atomDict[atomList[atom-1]][prop][0]
                else:
                    groupDict[prop][0] += atomDict[atomList[atom-1]]['quadContrib'][prop][0]
    groupDict.update({'|Mu_Intra|' : [math.sqrt(groupDict['Mu_Intra_X'][0]**2 + groupDict['Mu_Intra_Y'][0]**2 + groupDict['Mu_Intra_Z'][0]**2)]}) 
    groupDict.update({'|Mu_Bond|' : [math.sqrt(groupDict['Mu_Bond_X'][0]**2 + groupDict['Mu_Bond_Y'][0]**2 + groupDict['Mu_Bond_Z'][0]**2)]}) 
    groupDict.update({'|Mu_Bond|' : [math.sqrt(groupDict['Mu_X'][0]**2 + groupDict['Mu_Y'][0]**2 + groupDict['Mu_Z'][0]**2)]})     
    return groupDict

In [102]:
def extract_sub_props(sumFileNoExt,subAtoms,groupProps=True,bcpId = [[1,2]],lapRhoCpAtoms=[]):
    #returns a dictionary of all group properties - bcp, group, atomic, and vscc
    #sumFileNoExt - a sumfile name, without the .sum
    #subAtoms - indices of atoms in the molecule comprising group - starts at 1
    #groupProps - boolean - do you want to compute group properties
#    bcpIdx - list of 2 length lists. Pass empty list if no bcp properties wanted. 2 length lists are indices of atoms that you want BCPs for
    #lapRhoCpAtoms = list of atom indices that you want to find laprhoCPs for. Defaults to empty
    #vol surface is 0.001 au isodensity
    sumFile = open(sumFileNoExt+".sum","r") #open file, read lines, close file
    data = sumFile.readlines()
    sumFile.close()
    #find atom labels in atomList
    atomTable = get_table(data,'q(A)              L(A)              K(A)          K_Scaled(A)      |Mu_Intra(A)|')
    atomList = list(atomTable['Atom'])
    subatomLabels = []
    for i,atom in enumerate(atomList):
        if any(x == i+1 for x in subAtoms):
            subatomLabels.append(atomList[i])
    #if bcpIdx not empty, get bcp properties for each listed bcp
#right now every time I run, it passes the bcpIdx that it starts with to the next loop. Weird af
    bcpIdx = []
    #so here, code was doing something weird. bcpId was stored in memory and not reset to [[1,2]] every function call
        #every iteration updated to to -1 for each index, so I'm starting at [1,2] in bcpId and copying the values(
        #not pointers to bcpIdx)
        #don't know why it's necessary but it works now.
        #anyways that's why these 5 lines are here
    atomicProps = get_atomic_props(data) # get atomic dictionary
    for bcp in bcpId: 
        atList=[]
        for at in bcp:
            atList.append(at)
        bcpIdx.append(atList)    

    if len(bcpIdx) > 0:
        bcpProperties = {} #initialize empty dictionary
        for bcpPair in bcpIdx:
            bcpPair[0] -= 1 # lists start from 0, indices would be passed from molecule
            bcpPair[1] -= 1 # which would start at 1, so adjust index
            bcpBlock=[] #reset bcpBlock so won't be copied into future loop if block unable to be found
            bcpBlock = get_bcp_block(data, [atomList[i] for i in bcpPair])
            #add the bcp properties to the dictionary under the atom labels
            prop = get_bcp_properties(bcpBlock)
            prop.update({'DI(R,G)': [get_sub_di(data,subatomLabels)]})
            for key in atomicProps:
                if key == atomList[bcpPair[0]] or key == atomList[bcpPair[1]]:
                    if '1' in key:
                        keynum=1
                    elif '2' in key:
                        keynum=2
                    prop.update({'r(BCP-{at})'.format(at=keynum):[get_dist(atomicProps[key]['xyz'],prop['xyz'])]})
            bcpProperties.update({'{at1}-{at2}'.format(at1=atomList[bcpPair[0]],at2=atomList[bcpPair[1]]):prop })        
    else: #
        bcpProperties = "BCP Properties not requested"
    if groupProps: #if you want group properties
        for atom in atomicProps: #update atomic dictionary with quadrupole contribution
            atomicProps[atom].update({'quadContrib': get_atomic_quad_contrib(atomicProps[atom])})
        subDict = get_sub_props(atomicProps,subAtoms,atomList) # get substituent properties
    else:
        subDict = "Group Properties not requested"

    if len(lapRhoCpAtoms) > 0: #if we want laprhocps for at least one atom
        vsccProps={}
        for atom in lapRhoCpAtoms:#for each atom requested, get lapRhoCps
            allCC = get_cc_props(sumFileNoExt,atomList[atom-1])
            vsccProps.update({atomList[atom-1]: identify_vscc(allCC,atomicProps)})
    else:
        vsccProps={"VSCC Props not requested"}
    #create output dictionary to return all requested properties, if a property not requested return a string stating that    
    outDict = {'Group':subDict,'Atomic':atomicProps,'BCP':bcpProperties,'VSCC':vsccProps}    
    return outDict

In [103]:
extract_sub_props('SubH_CFHCH3_reor_wb97xd_aug-cc-pvtz',subAtoms=[1,3,4,5,6,7,8])

['C1', 'H2', '4.8650179565E+00', '9.1751823538E-01', '8.5123126074E+00', '4.6441045862E+01', '(Bonded)']
['C1', 'H3', '4.8650714703E+00', '9.1748698866E-01', '8.5120227147E+00', '4.6439133763E+01', '(Bonded)']
['H2', 'H3', '9.5810085587E-01', '3.5432942547E-02', '1.7934716133E+00', '1.7934588489E+00']
['C1', 'C4', '3.1643953903E+01', '9.8376748529E-01', '9.1269427080E+00', '8.2491649664E+00', '(Bonded)']
['H2', 'C4', '5.8699960744E+00', '4.0547710285E-02', '2.0523603786E+00', '3.4000386896E-01']
['H3', 'C4', '5.8700489208E+00', '4.0525862067E-02', '2.0512399115E+00', '3.3982066555E-01']
['C1', 'H5', '5.3260093287E+00', '4.2223207328E-02', '3.9172751691E-01', '2.1278370287E+00']
['H2', 'H5', '9.7804192368E-01', '4.0944423799E-03', '2.0724404051E-01', '2.0633927783E-01']
['H3', 'H5', '9.7413485843E-01', '1.1922523879E-02', '6.0346543118E-01', '6.0083516602E-01']
['C4', 'H5', '5.4368380502E+00', '9.5851942492E-01', '8.0374529326E+00', '4.8304552262E+01', '(Bonded)']
['C1', 'H6', '5.326049

{'Group': {'q': [-0.012042308080000064],
  'K': [177.45030553599003],
  'K_Scaled': [-178.44807629584],
  'Mu_Intra_X': [0.16653271575],
  'Mu_Intra_Y': [-0.41236599408389996],
  'Mu_Intra_Z': [-0.9758305059259],
  'Mu_Bond_X': [-0.47675302472190006],
  'Mu_Bond_Y': [0.6787360862049749],
  'Mu_Bond_Z': [1.68197961953125],
  'Mu_X': [-0.31022030898],
  'Mu_Y': [0.26637009212429996],
  'Mu_Z': [0.7061491135953],
  'Q_xx': [0.5211722027934232],
  'Q_xy': [0.404925215810474],
  'Q_xz': [0.23162618857614892],
  'Q_yy': [0.33104053675465406],
  'Q_yz': [-0.38996211284015003],
  'Q_zz': [-0.8522127395480739],
  'Vol': [423.130124775],
  '|Mu_Intra|': [1.0723917356952501],
  '|Mu_Bond|': [0.8159878900531444]},
 'Atomic': {'C1': {'xyz': array([0., 0., 0.]),
   'q': [0.6106406231],
   'K': [37.421323786],
   'K_Scaled': [-37.631737076],
   'Mu_Intra_X': [0.20513686803],
   'Mu_Intra_Y': [-0.28607647069],
   'Mu_Intra_Z': [-0.56196765211],
   'Mu_Bond_X': [-0.15692128852],
   'Mu_Bond_Y': [0.2192

In [104]:
def find_connected(data,negXAtomLabel,originAtomLabel):
    #find all atoms to which bonded
    bcpLines=[]
    for line in data:
        if 'Type = (3,-1)' in line and negXAtomLabel not in line and originAtomLabel in line:
            bcpLines.append(line)
    bcpList = []        
    for bcp in bcpLines:
        splitbcp = bcp.split()
        bcpList.append([splitbcp[4],splitbcp[5]])
    return bcpList

In [105]:
def get_bcp_dif(bcpBlockA,bcpBlockB):
    #get difference in bcp space between two bcps
    return

In [155]:
def get_bcp_reference(originAtom,numBonds,refDict):
    if originAtomNoNum =='C' and numBonds==4: #sp3 carbon
        retDict = refDict['C']['sp3']
    elif originAtomNoNum =='C' and numBonds==3: #sp2 carbon
        retDict = refDict['C']['sp2']
        #note linear shouldn't get to this point
    elif originAtomNoNum =='B' and numBonds==3: #planar boron
        print('planar boron')
    elif originAtomNoNum =='B' and numBonds==4: #sp3 boron
        print('sp3 boron')
    elif originAtomNoNum == 'N' and numBonds==4:
        print('ammonium')
    elif originAtomNoNum =='Al' and numBonds==3: #planar boron
        print('planar aluminum')
    elif originAtomNoNum =='Al' and numBonds==4: #sp3 boron
        print('sp3 aluminum') 
    elif originAtomNoNum =='Si' and numBonds==4: #sp3 carbon
        print('sp3 silicon')
    elif originAtomNoNum =='Si' and numBonds==3: #sp2 carbon
        print('sp2 silicon')
    elif originAtomNoNum == 'P' and numBonds==4:
        print('phosphonium')
    return retDict 

In [107]:
def find_bcp_match(data,originAtomXYZ,negXAtomLabel,originAtomLabel):
    #find best 1-1 match of bcps possible
    #note that there has to be some clockwise check or something
    #find bcps connected to origin atom that are not the -x axis atom
    originBCPs = find_connected(data,negXAtomLabel,originAtomLabel)
    bcpPropDict = {}
    #get the bcp properties
    for bcp in originBCPs:
        bcpBlock = get_bcp_block(data,atPair=bcp)
        bcpPropDict.update({bcp[0]+'-'+bcp[1]: get_bcp_properties(bcpBlock)})
    clockwiseKeys = find_clockwise_rot(bcpPropDict,originAtomXYZ,originAtomLabel)
    #at this point have bcpDictionary ordered from 1st to last with clockwise bcp
    #need to generate reference bcpDictionary clockwise and pairwise compare them
    #need different reference for different cases
    print(originAtomNoNum,numBonds)
    return clockwiseKeys

In [108]:
def angle_btw_bcp(xyzA,xyzB):
    #find angle between A and B
    #note - flattening to yz plane
    xyzA[0] = 0.0
    xyzB[0]=0.0
    angle= math.acos((xyzA[0]*xyzB[0]+xyzA[1]*xyzB[1]+xyzA[2]*xyzB[2])/(math.sqrt(xyzA[0]**2+xyzA[1]**2+xyzA[2]**2)*math.sqrt(xyzB[0]**2+xyzB[1]**2+xyzB[2]**2)))
    return angle

In [153]:
def find_clockwise_rot(bcpPropDict,originAtomXYZ,originAtomLabel):
    #given dictionary of bcp properties, find which ones are in a clockwise rotation
    #return list of dictionary keys ordered for clockwise rotation
    crossDict = {}
    for key1 in bcpPropDict:
        for key2 in bcpPropDict:
            if key1 != key2:
                xyz1 = bcpPropDict[key1]['xyz']
                xyz1[0] = 0.#project to yz plane
                xyz2=bcpPropDict[key2]['xyz']
                xyz2[0]=0.
                cross=np.cross(bcpPropDict[key1]['xyz'],bcpPropDict[key2]['xyz'])[0]
                if cross < 0:
                    isClockwise = True
                else:
                    isClockwise = False
                if isClockwise:
                    tempDict={'Is Clockwise':isClockwise,'cross':cross}
                    bondAt1 = key1.replace(originAtomLabel+'-','')
                    bondAt1 = bondAt1.replace('-'+originAtomLabel,'')
                    bondAt2 = key2.replace(originAtomLabel+'-','')
                    bondAt2 = bondAt2.replace('-'+originAtomLabel,'')
                    crossDict.update({bondAt1 + '-To-' + bondAt2:tempDict})
    orderDict= {}                
    for cw in crossDict:
        string = cw.split('-')
        start = string[0]
        end = string[2]
        orderDict.update({cw:{'Start':start,'End':end}})
#         keyDict={} #only one loop with key
#         xyz = bcpPropDict[key]['xyz']-originAtomXYZ
#         keyDict.update({'AngleToY': angle_btw_bcp(xyz,np.array([0.0,1.0,0.0]))})
#         keyDict.update({'AngleToZ': angle_btw_bcp(xyz,np.array([0.0,0.0,1.0]))})
#         angleDict.update({key:keyDict})
    keysList = list(orderDict.keys())
    if len(keysList) == 3:
        if orderDict[keysList[0]]['End'] != orderDict[keysList[1]]['Start'] and orderDict[keysList[0]]['End'] == orderDict[keysList[2]]['Start']:
            reordered_dict = {k : orderDict[k] for k in [keysList[0],keysList[2],keysList[1]]}
        else:
            reordered_dict = orderDict
        ordered_bcp_props = {}
        for order in reordered_dict:
            for bcp in bcpPropDict:
                print(bcp)
                if reordered_dict[order]['Start'] in bcp:
                    ordered_bcp_props.update({bcp:bcpPropDict[bcp]})
                    continue
    else:
        ordered_bcp_props = bcpPropDict
    return ordered_bcp_props

In [110]:
# sumFile = open('SubCH3_CFH2_anti2146_reor'+".sum","r") #open file, read lines, close file
# data = sumFile.readlines()
# sumFile.close()
# testProps = extract_sub_props('SubCH3_CFH2_anti2146_reor',subAtoms=[1,3,4,5,6,7,8])
# originAtomCoords =  testProps['Atomic']['C1']['xyz']
# find_bcp_match(data,originAtomCoords,"H2",'C1')

In [111]:
def is_clockwise(bcpXYZListA,bcpXYZListB):
    #check if a given map of bcps 'matches' eg match 1=4, 2=5, 3=6, make sure 1-2-3 rotate clockwise
    # and 4-5-6 also rotate clockwise
    return

In [112]:
def sub_prop_frame(csvFile):
    csvFrame = pd.read_csv(csvFile)
   
    #Take csv file for format and extract properties:
    #Substituent, subAtoms, label1,label2...
    #Substituent: string of substituent
    #subAtoms: string of space separated substituent atoms eg '1 3 4'
    #label1 - contains .sum file with no extension
    #labeli could be e.g. a model chemistry, or substrate
    #that is the column "SubH" would have sum files for the substituents attached to H
    #SubC6H5 would have sum files for substituents attached to C6H5 etc
    all_label_dict = {} #initialize output dictionary. Key structure: label:{Group,BCP}
    
    ncolumns = csvFrame.shape[1]
    nrow = csvFrame.shape[0]
    subAtoms = []
    for sub in range(0,nrow): #get subAtoms from csv in list format
        subAtomsString = csvFrame.loc[sub]['subAtoms'].split()
        subAtomsInt = [eval(i) for i in subAtomsString]
        subAtoms.append(subAtomsInt)
    for col in range(2,ncolumns): #2 is the start of the label columns, so for all labels:
        ind_label_dict = {} # create empty dictionary for this label
        count=0 #For first iteration for each label will create the data frame
        for row_num,sub in enumerate(csvFrame['Substituent']): #iterate over substituents
            #extract sumfile name from table
            sumFile = csvFrame[csvFrame['Substituent']==sub][csvFrame.columns[col]].iloc[0]
            #get properties
            extracted_props = extract_sub_props(sumFile,subAtoms[row_num])
            # dont want to store array in data frame
            excludeKeys = ['xyz']
            #add a substituent label to data frame
            extracted_props['Group'].update({'Substituent': sub})
            if count ==0:
                count = 1 #don't come here again for this label (don't need to make data frame again)
                #create data frame
                groupFrame = pd.DataFrame.from_dict(extracted_props['Group'],orient='columns')
                #for each bcp properties gotten for - currently only do this for one
                #should work generally soon
                for bnum,bcp in enumerate(extracted_props['BCP']): #currently only use for one bcp query
                    tempbcpDict = {k: extracted_props['BCP'][bcp][k] for k in set(list(extracted_props['BCP'][bcp].keys()))-set(excludeKeys)}
                    tempbcpDict.update({'x': [extracted_props['BCP'][bcp]['xyz'][0]]})
                    tempbcpDict.update({'y': [extracted_props['BCP'][bcp]['xyz'][1]]})
                    tempbcpDict.update({'z': [extracted_props['BCP'][bcp]['xyz'][2]]})
                    tempbcpDict.update({'Substituent' : sub})
                    tempbcpDict.update({'BCP': bcp})
                    if bnum ==0: #create data frame for first bcp in list
                        bcpFrame = pd.DataFrame.from_dict(tempbcpDict,orient='columns')
                    else: #else add to data frame
                        bcpFrame = pd.concat([bcpFrame,pd.DataFrame(tempbcpDict)],ignore_index=True)            
            else: #add to data frame after first iteration
                groupFrame = pd.concat([groupFrame,pd.DataFrame(extracted_props['Group'])],ignore_index=True)
                for bcp in extracted_props['BCP']:
                    tempbcpDict = {k: extracted_props['BCP'][bcp][k] for k in set(list(extracted_props['BCP'][bcp].keys()))-set(excludeKeys)}
                    tempbcpDict.update({'x': [extracted_props['BCP'][bcp]['xyz'][0]]})
                    tempbcpDict.update({'y': [extracted_props['BCP'][bcp]['xyz'][1]]})
                    tempbcpDict.update({'z': [extracted_props['BCP'][bcp]['xyz'][2]]})
                    tempbcpDict.update({'Substituent' : sub})
                    tempbcpDict.update({'BCP': bcp})
                    bcpFrame = pd.concat([bcpFrame,pd.DataFrame(tempbcpDict)],ignore_index=True)            
#create output dictioanry of data frames {label1:{Group,BCP},label2:{Group,BCP},....}
        ind_label_dict.update({'Group': groupFrame}) 
        ind_label_dict.update({'BCP': bcpFrame})
        all_label_dict.update({csvFrame.columns[col]: ind_label_dict })
    return all_label_dict            

In [113]:
def get_xyz(sumfile):
    sumFile = open(sumfile+".sum","r") #open file, read lines, close file
    data = sumFile.readlines()
    sumFile.close()
    xyzTable = get_table(data,'Charge                X                  Y                  Z',endString='Some Atomic Properties:')
    xyzTable['X'] = pd.to_numeric(xyzTable['X'],downcast='float')
    xyzTable['Y'] = pd.to_numeric(xyzTable['Y'],downcast='float')
    xyzTable['Z'] = pd.to_numeric(xyzTable['Z'],downcast='float')
    return {'xyz':xyzTable[['X','Y','Z']].to_numpy(),'Atoms':xyzTable['Atom']}

In [114]:
def set_origin(xyzArray,originAtom):
    org = xyzArray[originAtom-1,]
    return xyzArray - org

In [115]:
def get_lengths(xyzArray):
    lengths = np.array([])
    for atom in xyzArray:
        length=0
        for coord in atom:
            length += coord**2
        lengths = np.append(lengths,length)
    return lengths    

In [116]:
def zero_y_for_negx(t_xyz,negXAtom):
    if t_xyz[0,negXAtom-1] == 0 and t_xyz[1,negXAtom-1] == 0: #on xy
        print('hi')
        G= np.array([[0.,0.,1.],[0.,1.,0.],[-1.,0.,0.]])
    elif t_xyz[1,negXAtom-1] == 0 and t_xyz[2,negXAtom-1] == 0: #already on x axis atom 2 y and z=0
        print('here')
        G = np.array([[1.,0.,0.],[0.,1.,0.],[0.,0.,1.]])
    elif t_xyz[0,negXAtom-1] == 0 and t_xyz[2,negXAtom-1] == 0:
        print('ho there')
        G = np.array([[0.,1.,0.],[-1.,0.,0.],[0.,0.,1.]])
    else:
        print('no way')
        d = t_xyz[0,negXAtom-1]/math.sqrt(t_xyz[0,negXAtom-1]**2 + t_xyz[1,negXAtom-1]**2)
        s = t_xyz[1,negXAtom-1]/math.sqrt(t_xyz[0,negXAtom-1]**2 + t_xyz[1,negXAtom-1]**2)
        G = np.array([[d,s,0.],[-s,d,0.],[0.,0.,1.]])
    return np.matmul(G, t_xyz)    

In [117]:
def zero_z_for_negx(t_xyz,negXAtom):
    #perform after y
    d = t_xyz[0,negXAtom-1]/math.sqrt(t_xyz[0,negXAtom-1]**2 + t_xyz[2,negXAtom-1]**2)
    s = t_xyz[2,negXAtom-1]/math.sqrt(t_xyz[0,negXAtom-1]**2 + t_xyz[2,negXAtom-1]**2)
    G = np.array([[d,0,s],[0,1,0],[-s,0,d]])
    return np.matmul(G,t_xyz)

In [118]:
def set_xaxis(xyzArray,negXAtom):
    t_xyz = xyzArray.T
    
    #define initial xyz vector lengths. Should be unchanged after rotation
    tol = 0.0001 #tolerance for change
    initial_lengths= get_lengths(xyzArray)

    #Set Givens matrix for first rotation to zero [2,2]
#     if t_xyz[0,1] == 0 and t_xyz[1,1] == 0: #on xy
#         G= np.array([[0,0,1],[0,1,0],[-1,0,0]])
#     elif t_xyz[1,1] == 0 and t_xyz[2,1] == 0: #already on x axis atom 2 y and z=0
#         G = np.array([[1,0,0],[0,0,1],[0,0,1]])
#     elif t_xyz[0,1] == 0 and t_xyz[2,1] == 0:
#         G = np.array([[0,1,0],[-1,0,0],[0,0,1]])
#     else:
#         d = t_xyz[0,1]/math.sqrt(t_xyz[0,1]**2 + t_xyz[1,1]**2)
#         s = t_xyz[1,1]/math.sqrt(t_xyz[0,1]**2 + t_xyz[1,1]**2)
#         G = np.array([[d,s,0],[-s,d,0],[0,0,1]])
    t_rot1 = zero_y_for_negx(t_xyz,negXAtom)
    rot1_lengths = get_lengths(t_rot1.T)
    
    if np.any((rot1_lengths - initial_lengths)>tol):
        print("Geometry perturbation exceeded")
    t_rot2 = zero_z_for_negx(t_rot1,negXAtom)
    rot2_lengths = get_lengths(t_rot2.T)
    
    if np.any((rot2_lengths - initial_lengths)>tol):
        print("Geometry perturbation exceeded")
    if t_rot2[0,negXAtom-1] > 0: #if negxatom is along +x, rotate 180
        G = np.array([[-1,0,0],[0,1,0],[0,0,-1]])
        t_rot_final = np.matmul(G,t_rot2)
        return t_rot_final.T
    else:
        return t_rot2.T

In [ ]:
def align_dicts(testDict,refDict):
    testKeysList = list(testDict.keys())
    refDictKeysList = list(refDict.keys())
    dif00 = get_popelier_dif(testDict[testKeysList[0]],refDict[refDictKeysList[0]])
    dif11 = get_popelier_dif(testDict[testKeysList[1]],refDict[refDictKeysList[1]])
    dif22 = get_popelier_dif(testDict[testKeysList[2]],refDict[refDictKeysList[2]])
    dif10 = get_popelier_dif(testDict[testKeysList[1]],refDict[refDictKeysList[0]])
    dif21 = get_popelier_dif(testDict[testKeysList[2]],refDict[refDictKeysList[1]])
    dif02 = get_popelier_dif(testDict[testKeysList[0]],refDict[refDictKeysList[2]])
    dif20 = get_popelier_dif(testDict[testKeysList[2]],refDict[refDictKeysList[0]])
    dif01 = get_popelier_dif(testDict[testKeysList[0]],refDict[refDictKeysList[1]])
    dif12 = get_popelier_dif(testDict[testKeysList[1]],refDict[refDictKeysList[2]])
    matchScores = [dif00+dif11+dif22,dif10+dif21+dif02,dif20+dif01+dif12]
    minInd = matchScores.index(min(matchScores))
    
    if refDict[refDictKeysList[0]]['xyz'][1] > 0 and abs(refDict[refDictKeysList[0]]['xyz'][2]) < 0.01:
        refPosY=0
    elif refDict[refDictKeysList[1]]['xyz'][1] > 0 and abs(refDict[refDictKeysList[1]]['xyz'][2]) < 0.01:
        refPosY=1
    elif refDict[refDictKeysList[2]]['xyz'][1] > 0 and abs(refDict[refDictKeysList[2]]['xyz'][2]) < 0.01: 
        refPosY=2
    if minInd==0:
        if refPosY==0:
            posYPoint = testDict[testKeysList[0]]['xyz']
        elif refPosY == 1:
            posYPoint = testDict[testKeysList[1]]['xyz']
        elif refPosY == 2:
            posYPoint = testDict[testKeysList[2]]['xyz']
    elif minInd==1:
        if refPosY==0:
            posYPoint = testDict[testKeysList[1]]['xyz']
        elif refPosY == 1:
            posYPoint = testDict[testKeysList[2]]['xyz']
        elif refPosY == 2:
            posYPoint = testDict[testKeysList[0]]['xyz']
    elif minInd==2:
        if refPosY==0:
            posYPoint = testDict[testKeysList[2]]['xyz']
        elif refPosY == 1:
            posYPoint = testDict[testKeysList[0]]['xyz']
        elif refPosY == 2:
            posYPoint = testDict[testKeysList[1]]['xyz']
    return posYPoint    

In [119]:
def get_posy_point(sumFileNoExt,atomDict,attachedAtom,negXAtomLabel):
    
    ccProps = get_cc_props(sumFileNoExt,attachedAtom)    
    if len(ccProps) > 0:
        vscc = identify_vscc(ccProps,atomDict)
    else:
        vscc = {} 
    if len(vscc) == 1:
        #reorient setting vscc to +y
        vkeys = list(vscc.keys())
        posYPoint = vscc[vkeys[0]]['xyz']
    elif len(vscc) == 2 and "N" not in attachedAtom:
        vkeys = list(vscc.keys())
        
        posYPoint = (vscc[vkeys[0]]['xyz'] + vscc[vkeys[1]]['xyz'])/2
        print(posYPoint)
        #reorient to average of vscc points for +y
    else:
        refDict = get_reference_map()
        sumFile = open(sumFileNoExt+".sumviz",'r')
        data = sumFile.readlines()
        sumFile.close()
        bcpsToMatch = find_bcp_match(data,atomDict[attachedAtom]['xyz'],negXAtomLabel,attachedAtom,negXAtomLabel)
        numBonds=len(bcpsToMatch)+1
        atType = ''.join([i for i in attachedAtom if not i.isdigit()])
        matchDict = get_bcp_reference(atType,numBonds,refDict)
        posYPoint = align_dicts(matchDict,refDict)
        #reorient to another point
        #posy point will be the point that would lie along the y-axis in reference in maximal match case
        print('not done yet')
    return posYPoint    

In [120]:
def set_yaxis(xyzArray,posYArray):
    
    theta = math.atan2(posYArray[2],posYArray[1])
    print(theta)
    c = math.cos(theta)
    s = math.sin(theta)
    G = np.array([[1,0,0],[0,c,s],[0,-s,c]])
    rot1 = np.matmul(G,xyzArray.T)
    rot1vec = np.matmul(G,posYArray)
    
    if rot1vec[1] < 0:
        G2 = np.array([[1,0,0],[0,-1,0],[0,0,-1]])
        final_geom = np.matmul(G2,rot1)
    else:
        final_geom = rot1
    return final_geom.T    

In [121]:
def get_popelier_dif(bcpDictA,bcpDictB,statDict):
    distancesq = 0.
    for prop in bcpDictA:
        scaledA = (bcpDictA[prop]-statDict[prop]['mean'])/statDict[prop]['sd']
        scaledB = (bcpDictB[prop]-statDict[prop]['mean'])/statDict[prop]['sd']
        distancesq += (scaledA - scaledB)**2
    return math.sqrt(distancesq)    

In [122]:
def rotate_sheet():
    #extract the atomic and bcp properties
    
    #get sd and mean for each bcpproperty store in stat dict
    
    return

In [146]:
def get_ref_bcps(sumfilenoext,atPairList,originAtom,originAtomXYZ=np.array([0.,0.,0.])):
    sumFile = open(sumfilenoext+".sum","r") #open file, read lines, close file
    data = sumFile.readlines()
    sumFile.close()
    bcpDict = {}
    for bcp in atPairList:
        block = get_bcp_block(data,bcp)
        bcpDict.update({'{at1}-{at2}'.format(at1=bcp[0],at2=bcp[1]):get_bcp_properties(block)})
    clockbcpDict = find_clockwise_rot(bcpDict,originAtomXYZ,originAtom)    
    return clockbcpDict    

In [151]:
def get_reference_map():
    refDict={}
    carbonDict = {}
    carbonDict.update({'sp3':get_ref_bcps('SubH_CFHCH3_reor_wb97xd_aug-cc-pvtz',[['C1','H3'],['C1','C4'],['C1','F8']],originAtom='C1')})
    carbonDict.update({'sp2':get_ref_bcps('SubH_CHCH2-ReorJuly2-B3LYP-def2-TZVPPD-Field',[['C1','H3'],['C1','C4']],'C1')})
    refDict.update({'C':carbonDict})
    #update dictionary with further references
    return refDict

In [126]:
def rotate_substituent(sumFileNoExt,originAtom,negXAtom):
    molecule_xyz = get_xyz(sumFileNoExt)
    molecule_orig = set_origin(molecule_xyz['xyz'],originAtom)
    molecule_xaxis = set_xaxis(molecule_orig,negXAtom)
    negXAtomLabel = molecule_xyz['Atoms'][negXAtom-1]
    #next steps - reoreint based on atompic graph, start below, commented
    #update for efficiency later - getting all atomic properties for now because
    #thats how I wrote my vscc code
    sumFile = open(sumFileNoExt+".sum","r") #open file, read lines, close file
    data = sumFile.readlines()
    sumFile.close()
    atomDict = get_atomic_props(data)
    atomLabels = atomDict.keys()
    attachedAtom = molecule_xyz['Atoms'][originAtom-1]
    posYPoint = get_posy_point(sumFileNoExt,atomDict,attachedAtom,negXAtomLabel)
    final_orientation = set_yaxis(molecule_xaxis,posYPoint)
    outFrame = pd.DataFrame(final_orientation*0.529177,columns = ['x','y','z'])
    outFrame['Atom'] = molecule_xyz['Atoms']
    outFrame = outFrame[['Atom','x','y','z']]
    with open(sumFileNoExt+'reor.txt','w') as txt_file:
        of_string = outFrame.to_string(header=False,index=False)
        txt_file.write(of_string)
    return final_orientation

In [127]:
molecule = np.matmul(np.array([[1,0,0],[0,0,1],[0,-1,0]]),rotate_substituent("SubH_CFHCH3_wb97xd_aug-cc-pvtz",1,2).T).T
outFrame = pd.DataFrame(molecule*0.529177,columns = ['x','y','z'])
outFrame['Atom'] = ['C','H','H','C','H','H','H','F']
outFrame = outFrame[['Atom','x','y','z']]
with open('SubH_CFHCH3_'+'reor.txt','w') as txt_file:
    of_string = outFrame.to_string(header=False,index=False)
    txt_file.write(of_string)

no way


In [128]:
extract_sub_props('SubH_NHCH3-ReorPosY-B3LYP-def2-TZVPPD-Field',subAtoms=[1,3,4,5,6,7],lapRhoCpAtoms=[1])

['N1', 'H2', '4.9114849552E+00', '8.9047660967E-01', '5.5723514781E+00', '6.6412117699E+01', '(Bonded)']
['N1', 'H3', '4.9113986631E+00', '8.9046169418E-01', '5.5722581411E+00', '6.6412167597E+01', '(Bonded)']
['H2', 'H3', '4.3957749831E-01', '1.9747896495E-02', '1.4728063737E+00', '1.4728321502E+00']
['N1', 'C4', '4.4274647137E+01', '1.0240540940E+00', '6.4082417013E+00', '9.1347803249E+00', '(Bonded)']
['H2', 'C4', '3.7451720253E+00', '2.5365393183E-02', '1.8917616244E+00', '2.2626470218E-01']
['H3', 'C4', '3.7451111601E+00', '2.5355588581E-02', '1.8910634892E+00', '2.2617724304E-01']
['N1', 'H5', '8.0546533058E+00', '6.4025238775E-02', '4.0065188692E-01', '3.1630476019E+00']
['H2', 'H5', '6.7735823720E-01', '2.3173435851E-03', '1.7282845304E-01', '1.1448404113E-01']
['H3', 'H5', '6.7270314904E-01', '1.1603769991E-02', '8.6542916157E-01', '5.7326263120E-01']
['C4', 'H5', '5.2003807530E+00', '9.4517303489E-01', '8.4311445008E+00', '4.6694512330E+01', '(Bonded)']
['N1', 'H6', '8.054709

{'Group': {'q': [-0.32990173425999986],
  'K': [94.86610290335001],
  'K_Scaled': [-95.40208441798],
  'Mu_Intra_X': [-0.265613284434],
  'Mu_Intra_Y': [0.10662635503641997],
  'Mu_Intra_Z': [-0.24317947499220002],
  'Mu_Bond_X': [0.0563019005237],
  'Mu_Bond_Y': [0.15565805428114302],
  'Mu_Bond_Z': [0.66318701103144],
  'Mu_X': [-0.20931138391089998],
  'Mu_Y': [0.26228440930276004],
  'Mu_Z': [0.4200075360376],
  'Q_xx': [1.0622758443787461],
  'Q_xy': [0.7522418688527144],
  'Q_xz': [1.731823413154335],
  'Q_yy': [1.4547063341471924],
  'Q_yz': [-2.4310304862012684],
  'Q_zz': [-2.5169821785259394],
  'Vol': [368.195640525],
  '|Mu_Intra|': [0.3755740320012177],
  '|Mu_Bond|': [0.5375971513376627]},
 'Atomic': {'N1': {'xyz': array([0., 0., 0.]),
   'q': [-0.9901331884],
   'K': [54.845099907],
   'K_Scaled': [-55.154967803],
   'Mu_Intra_X': [-0.027766652808],
   'Mu_Intra_Y': [0.037426957843],
   'Mu_Intra_Z': [0.26083150469],
   'Mu_Bond_X': [-0.13847350268],
   'Mu_Bond_Y': [0.1

# Checking multipole rotational invariance

Herein, I check if and how a dipole changes upon rotation with no additional calculation. In this first block below, we extract the xyz coordinates of CH3CFH2 that has been rotated to the defined x-axis (atom 2 on -x). The 8-1-4-6 dihedral is equall to 180. There is also an additional conformer where the 2-1-4-6 dihedral is 180.

Visually analysing the structures, it is seen that if we zero the y-value of atom 3(index 2) by rotating around the x-axis that the structures will exactly overlap. I define angle phi and parameters c and s to construct a Givens rotation matrix to rotate the geometry to these coordinates. I write to a file to visually check that the geometries align basically exactly.

Extracting the dipole vector for group 1,3,4,5,6,7,8 and rotating that using the same rotation as the geometry underwent, we see what the dipole should be in this new geometry. Extracting the 2146 conformer dipole and comparing results in near identical dipole values.

Repeating the check with the quadrupole matrix however, is unsuccesful. Rotation of 8146 quadrupole matrix does not result in 2146 quadrupole matrix. Taking rotated 8146 geometry and explicitly calculating properties again does result in the 2146 quadrupole matrix as expected. Rotated 8146 and 2146 are essentially the same geometries.

## TLDR: can rotate dipoles, not quadrupoles

In [129]:
test_xyz = get_xyz("SubCH3_CFH2_anti8146_reor")
phi = math.atan(abs(test_xyz['xyz'][2,1])/abs(test_xyz['xyz'][2,2]))
c = math.cos(phi)
s = math.sin(phi)
G = np.array([[1,0,0],[0,c,s],[0,-s,c]])
outxyz = np.matmul(G,test_xyz['xyz'].T).T
outFrame = pd.DataFrame(outxyz*0.529177,columns = ['x','y','z'])
outFrame['Atom'] = test_xyz['Atoms']
with open('SubCH3_CFH2_anti8146_reoragin.txt','w') as txt_file:
    of_string = outFrame.to_string(header=False,index=False)
    txt_file.write(of_string)
testGroupProps = extract_sub_props("SubCH3_CFH2_anti8146_reor",[1,3,4,5,6,7,8])['Group']
muArray = np.array([testGroupProps['Mu_X'][0],testGroupProps['Mu_Y'][0],testGroupProps['Mu_Z'][0]])
np.matmul(G,muArray)

['C1', 'H2', '4.8849730179E+00', '9.1787065645E-01', '8.5269517098E+00', '4.6222153596E+01', '(Bonded)']
['C1', 'H3', '4.8849586142E+00', '9.1790726151E-01', '8.5272917682E+00', '4.6223963230E+01', '(Bonded)']
['H2', 'H3', '9.6790419332E-01', '3.5855766139E-02', '1.8056255728E+00', '1.8056242554E+00']
['C1', 'C4', '3.1595885216E+01', '9.8233660541E-01', '9.1258356918E+00', '8.2386903511E+00', '(Bonded)']
['H2', 'C4', '5.8988858214E+00', '4.0913255117E-02', '2.0603107299E+00', '3.4313252536E-01']
['H3', 'C4', '5.8988889861E+00', '4.0915563106E-02', '2.0604254524E+00', '3.4315188207E-01']
['C1', 'H5', '5.3307619098E+00', '4.2276371244E-02', '3.9274441724E-01', '2.1257754950E+00']
['H2', 'H5', '9.8524453771E-01', '4.1222362019E-03', '2.0758767431E-01', '2.0727769306E-01']
['H3', 'H5', '9.8125058177E-01', '1.2111588751E-02', '6.0991524586E-01', '6.0900493146E-01']
['C4', 'H5', '5.4486846012E+00', '9.5902030331E-01', '8.0431404835E+00', '4.8222252761E+01', '(Bonded)']
['C1', 'H6', '5.364141

array([-0.30880574, -0.6646714 , -0.25699394])

In [130]:
#other conformer:
test2GroupProps = extract_sub_props('SubCH3_CFH2_anti2146_reor',subAtoms=[1,3,4,5,6,7,8],bcpId=[])['Group']
np.array([test2GroupProps['Mu_X'][0],test2GroupProps['Mu_Y'][0],test2GroupProps['Mu_Z'][0]])

array([-0.30857977, -0.66448898, -0.25780634])

In [131]:
quad8146 = np.array([[testGroupProps['Q_xx'][0],testGroupProps['Q_xy'][0],testGroupProps['Q_xz'][0]],
                    [testGroupProps['Q_xy'][0],testGroupProps['Q_yy'][0],testGroupProps['Q_yz'][0]],
                    [testGroupProps['Q_xz'][0],testGroupProps['Q_yz'][0],testGroupProps['Q_zz'][0]]])
quad8146 #does not match quad2146 below. Rotationally variant

array([[ 0.47234011,  0.26451711, -0.28960657],
       [ 0.26451711,  0.25553229, -0.29079658],
       [-0.28960657, -0.29079658, -0.7278724 ]])

In [132]:
np.matmul(G,quad8146)

array([[ 0.47234011,  0.26451711, -0.28960657],
       [-0.14582305, -0.15080052, -0.78191261],
       [-0.36411112, -0.35653696, -0.05453027]])

In [133]:
quad2146 =  np.array([[test2GroupProps['Q_xx'][0],test2GroupProps['Q_xy'][0],test2GroupProps['Q_xz'][0]],
                    [test2GroupProps['Q_xy'][0],test2GroupProps['Q_yy'][0],test2GroupProps['Q_yz'][0]],
                    [test2GroupProps['Q_xz'][0],test2GroupProps['Q_yz'][0],test2GroupProps['Q_zz'][0]]])
quad2146 

array([[ 0.46944932, -0.13544219, -0.36306422],
       [-0.13544219, -0.76791755, -0.20601016],
       [-0.36306422, -0.20601016,  0.29846823]])

In [134]:
#Explicitly calculate 8146 properties in new conformer(dupe of 2146)
test8146again = extract_sub_props('SubCH3_CFH2_anti8146_reoragin',subAtoms=[1,3,4,5,6,7,8])['Group']

['C1', 'H2', '4.8847351546E+00', '9.1787349356E-01', '8.5273867023E+00', '4.6222126604E+01', '(Bonded)']
['C1', 'H3', '4.8847427797E+00', '9.1778362193E-01', '8.5265517614E+00', '4.6217923567E+01', '(Bonded)']
['H2', 'H3', '9.6790391684E-01', '3.5855605369E-02', '1.8056108412E+00', '1.8056234485E+00']
['C1', 'C4', '3.1593512601E+01', '9.8223930703E-01', '9.1253690885E+00', '8.2381012045E+00', '(Bonded)']
['H2', 'C4', '5.8987451325E+00', '4.0912099191E-02', '2.0602449485E+00', '3.4313228071E-01']
['H3', 'C4', '5.8987074649E+00', '4.0904776055E-02', '2.0598905538E+00', '3.4307086112E-01']
['C1', 'H5', '5.3304993363E+00', '4.2281007767E-02', '3.9280631364E-01', '2.1260101380E+00']
['H2', 'H5', '9.8524725514E-01', '4.1226595204E-03', '2.0760822884E-01', '2.0729912552E-01']
['H3', 'H5', '9.8124585202E-01', '1.2111678548E-02', '6.0992223006E-01', '6.0900987800E-01']
['C4', 'H5', '5.4485580554E+00', '9.5893847221E-01', '8.0426756763E+00', '4.8218172209E+01', '(Bonded)']
['C1', 'H6', '5.363869

In [135]:
np.array([[test8146again['Q_xx'][0],test8146again['Q_xy'][0],test8146again['Q_xz'][0]],
                    [test8146again['Q_xy'][0],test8146again['Q_yy'][0],test8146again['Q_yz'][0]],
                    [test8146again['Q_xz'][0],test8146again['Q_yz'][0],test8146again['Q_zz'][0]]])
#Properties in new conformer 2146 are the same as original 2146

array([[ 0.46957594, -0.1355025 , -0.36310022],
       [-0.1355025 , -0.76865137, -0.20654642],
       [-0.36310022, -0.20654642,  0.29907543]])